<a href="https://colab.research.google.com/github/RahulArya15/Caption-Generator/blob/main/caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir ~/.kaggle/

In [ ]:
!cp kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hsankesara/flickr-image-dataset


100% 8.15G/8.16G [01:02<00:00, 129MB/s]
100% 8.16G/8.16G [01:02<00:00, 140MB/s]


In [ ]:
! unzip flickr-image-dataset.zip

In [ ]:
import string
import numpy as np
import pandas as pd
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.utils import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from tensorflow.keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
# small library for seeing the progress of loops.
from tqdm.notebook import tqdm #this was changed
tqdm().pandas()

0it [00:00, ?it/s]

In [ ]:
df = pd.read_csv('/content/flickr30k_images/results.csv', on_bad_lines='skip')
#print(df.head())
print(df.iloc[:,'image_name'])

KeyError: ignored

In [ ]:
#loading a text file into memory
def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text

In [ ]:
#get all images with their captions
def all_img_captions(filename):
  file = load_doc(filename)
  captions = file.split('\n')
  descriptions = {}
  for caption in captions[:-1]:
    img, caption = caption.split('\t')
    if img[:-2] not in descriptions:
      descriptions[img[:-2]] = [caption]
    else:
      descriptions[img[:-2]].append(caption)
  return descriptions


In [ ]:
#lower casing, removinf puntutations and words containing numbers
def cleaning_text(captions):
  table = str.maketrans('','', string.punctuation)
  for img,caps in captions.items():
    for i,img_caption in enumerate(caps):
      img_caption.replace("-"," ")
      desc = img_caption.split()

      #convers to lowercase
      desc = [word.lower() for word in desc]
      #remove punctuatin from each token
      desc = [word.translate(table) for word in desc]
      #removing 's and a
      desc = [word for word in desc if(len(word) > 1)]
      #remove tokens with numbers in them
      desc = [word for word in desc if(word.isalpha())]
      #convert back to string
      img_caption = ' '.join(desc)
      captions[img][i] = img_caption
  return captions

In [ ]:
#build vocabulary of all unique words
def text_vocalbulary(descriptions):
  vocab = set()
  for key in descriptions.key() :
    [vocab.update(d.split()) for d in descriptions[key]]
  return vocab

In [ ]:
#all description in one file
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key + '\t' + desc)
  data = "\n".join(lines)
  file = open(filename,"w")
  file.write(data)
  file.close()

In [ ]:
dataset_text = "/content/captions.txt"
dataset_images = "/content/Images"

In [ ]:
#we prepare our text data
filename = dataset_text #+ "/" + "Flickr30k.token.txt"
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)
#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving each description to file 
save_descriptions(clean_descriptions, "descriptions.txt")

FileNotFoundError: ignored